In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
##import Bedrock_Client
##import pyodbc
import pandas as pd
import SHMI_PREDICTOR_SUPERVISED

In [2]:
##df = pd.read_sql_query('select * fROM ref.SHMI_Model_Predict_Statistics',Bedrock_Conn)
import Pickle_Utility
df =Pickle_Utility.read('shimi_predictor')
df = df.sort_values(by = ['PREDICTOR','DIAGNOSIS_GROUP','Date'])

In [3]:
time_index = pd.DataFrame(pd.date_range('2019-05-01', periods=11, freq='MS')+ pd.DateOffset(days=12), columns=['Date'])

In [4]:
TEST= df.loc[(df.loc[:,'PREDICTOR']=='AGE_GROUP_3')&(df.loc[:,'DIAGNOSIS_GROUP']=='103'),:]

In [5]:
TEST

,INDICATOR_CODE,DIAGNOSIS_GROUP,PREDICTOR,PARAMETER_ESTIMATE,STANDARD_ERROR,WALD_CHI_SQUARE,WALD_DF,WALD_P_VALUE,PUBLISHED_DATE,SHMIPREDICTOR,Date
64146,I00699,103,AGE_GROUP_3,-2.5426,1.0032,6.4239,1,0.0113,20190513,201801201812,2019-05-13
57545,I00699,103,AGE_GROUP_3,-2.5039,1.0033,6.2291,1,0.0126,20190613,201802201901,2019-06-13
50945,I00699,103,AGE_GROUP_3,-2.4549,1.0033,5.9865,1,0.0144,20190713,201803201902,2019-07-13
44344,I00699,103,AGE_GROUP_3,-2.446,1.0034,5.9429,1,0.0148,20190813,201804201903,2019-08-13
37744,I00699,103,AGE_GROUP_3,-2.4213,1.0034,5.8226,1,0.0158,20190913,201805201904,2019-09-13
31145,I00699,103,AGE_GROUP_3,-2.3932,1.0035,5.6877,1,0.0171,20191013,201806201905,2019-10-13
24546,I00699,103,AGE_GROUP_3,-2.3733,1.0035,5.5929,1,0.018,20191113,201807201906,2019-11-13
17949,I00699,103,AGE_GROUP_3,-14.0346,341.0284,0.0017,1,0.9672,20191213,201808201907,2019-12-13
11351,I00699,103,AGE_GROUP_3,-14.0167,339.8537,0.0017,1,0.9671,20200113,201809201908,2020-01-13
4755,I00699,103,AGE_GROUP_3,-13.9166,315.6272,0.0019,1,0.9648,20200213,201810201909,2020-02-13


In [6]:
extract = pd.merge(time_index, TEST, left_on='Date',right_on='Date',how='left')
extract.set_index('Date',inplace=True)
extract['PARAMETER_ESTIMATE'] = extract['PARAMETER_ESTIMATE'].astype(float).interpolate()
extract['PARAMETER_ESTIMATE'] = extract['PARAMETER_ESTIMATE'].astype(float).interpolate(limit_direction ='backward')

In [7]:
extract

,INDICATOR_CODE,DIAGNOSIS_GROUP,PREDICTOR,PARAMETER_ESTIMATE,STANDARD_ERROR,WALD_CHI_SQUARE,WALD_DF,WALD_P_VALUE,PUBLISHED_DATE,SHMIPREDICTOR
Date,,,,,,,,,,
2019-05-13,I00699,103,AGE_GROUP_3,-2.5426,1.0032,6.4239,1,0.0113,20190513,201801201812
2019-06-13,I00699,103,AGE_GROUP_3,-2.5039,1.0033,6.2291,1,0.0126,20190613,201802201901
2019-07-13,I00699,103,AGE_GROUP_3,-2.4549,1.0033,5.9865,1,0.0144,20190713,201803201902
2019-08-13,I00699,103,AGE_GROUP_3,-2.4460,1.0034,5.9429,1,0.0148,20190813,201804201903
2019-09-13,I00699,103,AGE_GROUP_3,-2.4213,1.0034,5.8226,1,0.0158,20190913,201805201904
2019-10-13,I00699,103,AGE_GROUP_3,-2.3932,1.0035,5.6877,1,0.0171,20191013,201806201905
2019-11-13,I00699,103,AGE_GROUP_3,-2.3733,1.0035,5.5929,1,0.018,20191113,201807201906
2019-12-13,I00699,103,AGE_GROUP_3,-14.0346,341.0284,0.0017,1,0.9672,20191213,201808201907
2020-01-13,I00699,103,AGE_GROUP_3,-14.0167,339.8537,0.0017,1,0.9671,20200113,201809201908


In [8]:
chunks = SHMI_PREDICTOR_SUPERVISED.to_chunks(df, 'PREDICTOR','DIAGNOSIS_GROUP')

In [97]:
import numpy as np
test = np.array(SHMI_PREDICTOR_SUPERVISED.variable_to_series(chunks[0],3)).astype(float)

In [98]:
SHMI_PREDICTOR_SUPERVISED.supervised_for_steps(test,4,1)

(array([[-1.3545, -1.5674, -1.579 , -1.5918, -1.6894],
        [-1.5674, -1.579 , -1.5918, -1.6894, -1.8427],
        [-1.579 , -1.5918, -1.6894, -1.8427, -1.8411],
        [-1.5918, -1.6894, -1.8427, -1.8411, -1.8507],
        [-1.6894, -1.8427, -1.8411, -1.8507, -1.8199],
        [-1.8427, -1.8411, -1.8507, -1.8199, -1.994 ]]),
 array([-1.8411, -1.8507, -1.8199, -1.994 , -1.9773]))

In [99]:
test

array([-1.3545, -1.5674, -1.579 , -1.5918, -1.6894, -1.8427, -1.8411,
       -1.8507, -1.8199, -1.994 , -1.9773])

In [101]:
train_data, test_data,actual = SHMI_PREDICTOR_SUPERVISED.data_prep(chunks,4)

In [102]:
train_data.shape

(6605, 1, 1, 6, 5)

In [103]:
test_data.shape

(6605, 1, 5)

In [104]:
model_list = SHMI_PREDICTOR_SUPERVISED.get_models()

Defined 8 models


In [110]:
SHMI_PREDICTOR_SUPERVISED.evaluate_models(model_list,train_data,test_data,actual)

lr: 1.993 MAE
lasso: 2.584 MAE
ridge: 2.242 MAE
en: 2.584 MAE
huber: 1.540 MAE
llars: 2.584 MAE
pa: 0.192 MAE
sgd: 0.194 MAE


In [68]:
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import SGDRegressor
ml = HuberRegressor()
test,ab = SHMI_PREDICTOR_SUPERVISED.make_future_prediction(ml,df,steps=6,n_lag=4)

In [69]:
test['PREDICTOR'].nunique(),df['PREDICTOR'].nunique()

(52, 52)

In [70]:
df['PARAMETER_ESTIMATE'].astype(float).describe()

count    72591.000000
mean        -0.714975
std          3.485565
min        -22.379100
25%         -0.405400
50%          0.000000
75%          0.168000
max         22.067600
Name: PARAMETER_ESTIMATE, dtype: float64

In [71]:
test.describe()

,PARAMETER_ESTIMATE
count,39630.000000
mean,-0.730459
std,3.592557
min,-46.666371
25%,-0.417646
50%,0.000000
75%,0.177506
max,60.204308


In [72]:
test[test['PARAMETER_ESTIMATE']==test['PARAMETER_ESTIMATE'].min()]

,INDICATOR_CODE,DIAGNOSIS_GROUP,PREDICTOR,PARAMETER_ESTIMATE,STANDARD_ERROR,WALD_CHI_SQUARE,WALD_DF,WALD_P_VALUE,PUBLISHED_DATE,SHMIPREDICTOR
24563,FUTURE,139,AGE_GROUP_3,-46.666371,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201905202004


In [73]:
predictor = 'ADMISSION_MONTH_1'
diagnosis_group = '1'
df.loc[(df.loc[:,'PREDICTOR']==predictor)&(df.loc[:,'DIAGNOSIS_GROUP']==diagnosis_group),:]

,INDICATOR_CODE,DIAGNOSIS_GROUP,PREDICTOR,PARAMETER_ESTIMATE,STANDARD_ERROR,WALD_CHI_SQUARE,WALD_DF,WALD_P_VALUE,PUBLISHED_DATE,SHMIPREDICTOR,Date
59396,I00699,1,ADMISSION_MONTH_1,-0.2517,0.3737,0.4537,1,0.5006,20190513,201801201812,2019-05-13
52795,I00699,1,ADMISSION_MONTH_1,-0.1005,0.3654,0.0757,1,0.7832,20190613,201802201901,2019-06-13
46194,I00699,1,ADMISSION_MONTH_1,-0.0261,0.3502,0.0055,1,0.9407,20190713,201803201902,2019-07-13
39593,I00699,1,ADMISSION_MONTH_1,-0.0656,0.354,0.0343,1,0.853,20190813,201804201903,2019-08-13
32994,I00699,1,ADMISSION_MONTH_1,-0.0437,0.3523,0.0154,1,0.9013,20190913,201805201904,2019-09-13
26395,I00699,1,ADMISSION_MONTH_1,0.0700,0.3708,0.0357,1,0.8502,20191013,201806201905,2019-10-13
19796,I00699,1,ADMISSION_MONTH_1,0.0772,0.371,0.0433,1,0.8352,20191113,201807201906,2019-11-13
13198,I00699,1,ADMISSION_MONTH_1,0.1615,0.3672,0.1935,1,0.66,20191213,201808201907,2019-12-13
6601,I00699,1,ADMISSION_MONTH_1,0.1570,0.3728,0.1774,1,0.6736,20200113,201809201908,2020-01-13
3,I00699,1,ADMISSION_MONTH_1,0.0992,0.3674,0.0729,1,0.7871,20200213,201810201909,2020-02-13


In [74]:
test.loc[(test.loc[:,'PREDICTOR']==predictor)&(test.loc[:,'DIAGNOSIS_GROUP']==diagnosis_group),:]

,INDICATOR_CODE,DIAGNOSIS_GROUP,PREDICTOR,PARAMETER_ESTIMATE,STANDARD_ERROR,WALD_CHI_SQUARE,WALD_DF,WALD_P_VALUE,PUBLISHED_DATE,SHMIPREDICTOR
2556,FUTURE,1,ADMISSION_MONTH_1,0.119537,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201812201911
2557,FUTURE,1,ADMISSION_MONTH_1,0.108831,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201901201912
2558,FUTURE,1,ADMISSION_MONTH_1,0.091427,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201902202001
2559,FUTURE,1,ADMISSION_MONTH_1,0.094831,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201903202002
2560,FUTURE,1,ADMISSION_MONTH_1,0.100092,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201904202003
2561,FUTURE,1,ADMISSION_MONTH_1,0.094292,FUTURE,FUTURE,FUTURE,FUTURE,20200327,201905202004


In [75]:
test.to_csv('test.csv',index=False)

In [24]:
ab

,Date,SHIMIPREDICTOR,Predictor,Diagnosis_Group,PRED_DIFF,Average_Diff,New_Pred,Pre_Predict
0,2020-04-13,201812201911,ADMIMETH_1,126,[0.076375],0.076375,[-0.031024999999999997],[-0.24001345665087306]
1,2020-05-13,201901201912,ADMIMETH_1,126,[0.10554000000000001],0.10554,[0.07451500000000001],[-0.19223591778283852]
2,2020-06-13,201902202001,ADMIMETH_1,126,[0.10067916666666664],0.100679,[0.17519416666666665],[-0.14535107234201106]
3,2020-07-13,201903202002,ADMIMETH_1,126,[0.1013735714285714],0.101374,[0.27656773809523805],[-0.0976868617074271]
4,2020-08-13,201904202003,ADMIMETH_1,126,[0.10128677083333332],0.101287,[0.37785450892857136],[-0.055810663416984846]
5,2020-09-13,201905202004,ADMIMETH_1,126,[0.10129641534391537],0.101296,[0.47915092427248673],[-0.0112957978037284]
6,2020-04-13,201812201911,ADMIMETH_1,132,[0.029799999999999993],-0.0298,[0.40650000000000003],[0.16792447725822393]
7,2020-05-13,201901201912,ADMIMETH_1,132,[0.029799999999999993],-0.0298,[0.37670000000000003],[0.1980100696219181]
8,2020-04-13,201812201911,ADMIMETH_1,137,[0.008666666666666656],0.00866667,[-0.16973333333333335],[-0.03065773726135803]
9,2020-05-13,201901201912,ADMIMETH_1,137,[0.008666666666666656],0.00866667,[-0.1610666666666667],[-0.037860632003956264]


In [25]:
time_index = pd.DataFrame(pd.date_range('2019-05-01', periods=11, freq='MS')+ pd.DateOffset(days=12), columns=['Date'])
future_time_index = pd.DataFrame(pd.date_range(time_index['Date'].iloc[-1], periods=3, freq='MS')+ pd.DateOffset(days=12), columns=['Date'])


In [26]:
df2 = df
df2['PARAMETER_ESTIMATE'] =df['PARAMETER_ESTIMATE'].astype(float)
df2.groupby(['SHMIPREDICTOR','PREDICTOR','DIAGNOSIS_GROUP'])['PARAMETER_ESTIMATE'].describe().to_csv('historytrend.csv')

In [27]:
test.groupby(['SHMIPREDICTOR','PREDICTOR','DIAGNOSIS_GROUP'])['PARAMETER_ESTIMATE'].describe().to_csv('predictedtrend.csv')

In [28]:
from datetime import date

today = date.today()
print("Today's date:", today.strftime("%Y%m%d"))

Today's date: 20200327


In [29]:
int(df['SHMIPREDICTOR'].unique()[-1][0:4])

2018

In [30]:
import matplotlib.pyplot as plt
PREDICTOR = 'AGE_GROUP_14'
DIAGNOSIS_GROUP = '2'

new_df = df[(df['PREDICTOR']==PREDICTOR)&(df['DIAGNOSIS_GROUP']==DIAGNOSIS_GROUP)][['Date','PARAMETER_ESTIMATE']].sort_values(by='Date')


In [31]:
new_df2 = df[(df['PREDICTOR']==PREDICTOR)][['Date','PARAMETER_ESTIMATE']].sort_values(by='Date')
new_df2['PARAMETER_ESTIMATE'] = new_df2['PARAMETER_ESTIMATE'].astype(float)

In [32]:
new_df2.groupby('Date')['PARAMETER_ESTIMATE'].mean()

Date
2019-05-13    0.577468
2019-06-13    0.571559
2019-07-13    0.562301
2019-08-13    0.563252
2019-09-13    0.569328
2019-10-13    0.561574
2019-11-13    0.581050
2019-12-13    0.570097
2020-01-13    0.568148
2020-02-13    0.572361
2020-03-13    0.569407
Name: PARAMETER_ESTIMATE, dtype: float64

In [33]:
new_df.set_index('Date', inplace=True)

In [34]:
#new_df = pd.DataFrame(new_df2.groupby('Date')['PARAMETER_ESTIMATE'].mean())

In [35]:
new_df.astype('float')

,PARAMETER_ESTIMATE
Date,
2019-05-13,-0.8306
2019-06-13,-0.8145
2019-07-13,-0.8098
2019-08-13,-0.8014
2019-09-13,-0.7887
2019-10-13,-0.7856
2019-11-13,-0.7801
2019-12-13,-0.7803
2020-01-13,-0.7784


In [36]:
from numpy import array
def supervised_for_steps(series, n_lag, lead_time):
    data = list()
    for i in range(n_lag, len(series)):
        end_ix = i + (lead_time - 1)
        if end_ix >= len(series):
            break
        start_ix = i - n_lag
        ne = []
        for item in series[start_ix:i]:
            ne.append(item[0])
        ne.append([series[end_ix]][0][0])        
        row = ne        
        data.append(row)
    return array(data)

In [37]:
supervised_for_steps(new_df.astype('float').values, 2, 1)

array([[-0.8306, -0.8145, -0.8098],
       [-0.8145, -0.8098, -0.8014],
       [-0.8098, -0.8014, -0.7887],
       [-0.8014, -0.7887, -0.7856],
       [-0.7887, -0.7856, -0.7801],
       [-0.7856, -0.7801, -0.7803],
       [-0.7801, -0.7803, -0.7784],
       [-0.7803, -0.7784, -0.7692],
       [-0.7784, -0.7692, -0.7638]])

In [38]:
dataset = supervised_for_steps(new_df.astype('float').values, 1, 1)

In [39]:
n_lag = 1
train_data, test_data,validate_data = dataset[:-2], dataset[-2:-1], dataset[-1:]
print(train_data.shape, test_data.shape,validate_data.shape)

(8, 2) (1, 2) (1, 2)


In [40]:
models = SHMI_PREDICTOR_SUPERVISED.get_models()

Defined 8 models


In [41]:
train_data

array([[-0.8306, -0.8145],
       [-0.8145, -0.8098],
       [-0.8098, -0.8014],
       [-0.8014, -0.7887],
       [-0.7887, -0.7856],
       [-0.7856, -0.7801],
       [-0.7801, -0.7803],
       [-0.7803, -0.7784]])

In [42]:
#SHMI_PREDICTOR_SUPERVISED.evaluate_models(models, train_data, test_data, validate_data)

In [43]:
X, y = train_data[:,:-1],train_data[:,-1]

In [44]:
train_data.shape[1]

2

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

n_trees = 5
test = DecisionTreeRegressor()

In [46]:
import numpy as np
X_1 = np.concatenate((X, test_data[:,:-1]))
y_1 = np.concatenate((y, test_data[:,-1]))

In [47]:
lr = SHMI_PREDICTOR_SUPERVISED.fit_model(test, X, y)

In [48]:
test_data[:,:-1]

array([[-0.7784]])

In [49]:
lr.predict(test_data[:,:-1]),test_data[:,-1]

(array([-0.7803]), array([-0.7692]))

In [50]:
lr.predict(validate_data[:,:-1]),validate_data[:,-1]


(array([-0.7803]), array([-0.7638]))

In [51]:
lr2 = SHMI_PREDICTOR_SUPERVISED.fit_model(test, X_1, y_1)

In [52]:
lr2.predict(validate_data[:,:-1]),validate_data[:,-1]

(array([-0.7692]), array([-0.7638]))

In [53]:
tt = [[list() for _ in range(train_data.shape[1])] for _ in range(train_data.shape[0])]

In [54]:
tt[1][0]

[]

In [55]:
range(train_data.shape[1])

range(0, 2)

In [56]:
range(train_data.shape[0])

range(0, 8)

In [57]:
SHMI_PREDICTOR_SUPERVISED.evaluate_models(models,train_data[0],test_data[0],dataset[-2:-1][0][2])

IndexError: index 2 is out of bounds for axis 0 with size 2

In [58]:
train_data[0].shape[1]

IndexError: tuple index out of range

In [59]:
train_data

array([[-0.8306, -0.8145],
       [-0.8145, -0.8098],
       [-0.8098, -0.8014],
       [-0.8014, -0.7887],
       [-0.7887, -0.7856],
       [-0.7856, -0.7801],
       [-0.7801, -0.7803],
       [-0.7803, -0.7784]])